# 1. Giới thiệu
* Là một thuật toán thuộc nhóm Supervised Learning dc sử dụng cho cả classification và regression.
* Decision tree dc sử dụng để phát triển các model dự đoán và phân loại trong nh lĩnh vực khác nhau như:
  * Business Management
  * Customer Relationship Management
  * Fraudulent Statemant Detection
  * Engineering
  * Energy Consumption
  * Fault Diagnosis
  * Healthcare Management
  * Agriculture
  * ...
* Decision Tree thường dc sử dụng trog nghiên cứu hoạt động, đặc biệt trong phân tích quyết định, giúp xác định chiến lược có khả năng đạt dc mục tiêu tốt nhất.

# 2. Thuật toán
* Tìm các tính năng _(feature)_ mô tả có chứa thông tin nhất về target feature và sau đó chia tập dữ liệu dọc theo các giá trị của các feature này sao cho các giá trị target feature cho các tập con _(sub dataset)_ càng thuần khiết càng tốt.

$\Rightarrow$ Các tính năng mô tả dẫn đến target feature thuần thiết nhất dc gọi là có thông tin nhất.